# Groundwater Level Dossier

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import brodata

We can download the data of one Groundwater Level Dossier, using the class method `from_bro_id`. This returns a GroundwaterLevelDossier-object that contains all data from the downloaded xml-file.

In [ ]:
gld = brodata.gld.GroundwaterLevelDossier.from_bro_id("GLD000000012893")
gld

We can look at the contents of this GroundwaterLevelDossier by using the `to_dict()` method.

In [ ]:
gld.to_dict()

The observation-data is included in the `observation` attribute of the GroundwaterLevelDossier object, as a pandas DataFrame.

In [ ]:
df = gld.observation
df

We can generate a plot of the time-series with a different color for each meta-data. We define a method `plot_series` and call this method.

In [ ]:
def plot_series(df):
    f, ax = plt.subplots(figsize=(10, 8))
    columns = ["qualifier", "status", "observation_type"]
    for qualifier, status, observation_type in df[columns].drop_duplicates().values:
        mask = df["qualifier"] == qualifier
        if pd.isna(status):
            mask = mask & pd.isna(df["status"])
        else:
            mask = mask & (df["status"] == status)
        mask = mask & (df["observation_type"] == observation_type)
        label = f"{observation_type} {status} {qualifier}"
        if mask.sum() > 100:
            linestyle = "-"
            marker = None
        else:
            linestyle = "none"
            marker = "o"
        df.loc[mask, "value"].plot(label=label, linestyle=linestyle, marker=marker)
    ax.legend()
    return f, ax


plot_series(df);

## Multiple objects

### All measurements of one tube
We can download the data of all GroundwaterLevelDossiers of one perticular tube (piezometer) using `brodata.gmw.get_tube_observations`.

In [ ]:
df = brodata.gmw.get_tube_observations("GMW000000017757", 1)

We can plot the resulting pandas DataFrame gaian with the method `plot_series` we defined above.

In [ ]:
plot_series(df);

### All measurements of one well
We can download all meusurementd of one Groundwater Monitoring Well using the method `nlmod.gmw.get_observations`. This returns a pandas DataFrame with a feature (row) for each Groundwater Level Dossier thta contains measurements for each of the tubes of thhis Groundwater Monitoring Well.

In [ ]:
gdf = brodata.gmw.get_observations("GMW000000017757")
gdf = gdf.set_index(["groundwaterMonitoringWell", "tubeNumber"]).sort_index()

In [ ]:
gdf

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
for index in gdf.index.unique():
    observations = [x for x in gdf.loc[[index], "observation"] if not x.empty]
    df = pd.concat(observations).sort_index()
    for qualifier in df["qualifier"].unique():
        if pd.isna(qualifier):
            continue
        mask = df["qualifier"] == qualifier
        label = f"{index[0]}_{index[1]} {qualifier}"
        if mask.sum() > 100:
            linestyle = "-"
            marker = None
        else:
            linestyle = "none"
            marker = "o"
        df.loc[mask, "value"].plot(label=label, linestyle=linestyle, marker=marker)
plt.legend();

### All measurements within extent
You can download all groundwater level data within an extent using the method `brodata.gmw.get_data_in_extent`. This returns a GeoDataFrame.

In [ ]:
extent = [118200, 118400, 439700, 440000]
gdf = brodata.gmw.get_data_in_extent(
    extent=extent, kind="gld", combine=True, as_csv=True
)

In [ ]:
gdf.T

In [ ]:
gdf['observation'].iloc[0]

We can plot the locations within the GeoDataFrame on a map using the general `plot` method of this GeoDataFrame.

In [ ]:
gdf.plot()

## Observations summary
With the `brodata.gld.get_observations_summary` we can download a summary of te different observations within a Groundwater Level Dossier.

In [ ]:
brodata.gld.get_observations_summary("GLD000000012893")

## Objects as csv

In [ ]:
df = brodata.gld.get_objects_as_csv("GLD000000012893")
df

In [ ]:
f, ax = plt.subplots()
for qualifier in df["qualifier"].unique():
    if pd.isna(qualifier):
        continue
    mask = df["qualifier"] == qualifier
    df.loc[mask, "value"].plot(label=qualifier)
ax.legend();

## Series as csv

In [ ]:
df = brodata.gld.get_series_as_csv("GLD000000012893")
df